CPace sage notebook for generating test vectors for the CPace cipher suites.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive straight-line code from the Elligator paper, checked also against the code from the hash_to_curve team.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
5.) Implementation and test vector generation for the AuCPace-specific constructions for generator
    calculation and SK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 9.0 on ubuntu linux.

In [1]:
import sys
from importlib import reload

sys.path.append("sagelib")

from CPace_hashing import *
from CPace_testvectors import *
from CPace_string_utils import *
from RFC7748_X448_X25519 import *
from CPace_montgomery import *

from hash_to_field import I2OSP, OS2IP
from suite_p256 import *
from suite_p384 import *
from suite_p521 import *

from CPace_weierstrass import *
from CPace_coffee import *


In [18]:
def prepend_len(data):
    "LEB128 encoding"
    length = len(data)
    length_encoded = b""
    while length > 0:
        if length < 128:
            length_encoded += bytes([length])
        else:
            length_encoded += bytes([(length & 0x7f) + 0x80])
        length = int(length >> 7)
        print (length)
    return length_encoded + data 



128
1
0


b'\x81\x80\x0100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
# 2.) Definitions for the X25519 test cases



In [ ]:
output_test_vectors_for_weak_points_255()

In [ ]:
    
output_test_vectors_for_weak_points_448()

In [ ]:
    
generate_testvectors_string_functions()

In [ ]:
# 4.) Definitions of the Elligator2 test cases
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#
G = G_X25519()

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = decodeUCoordinate(G.elligator2(etc1_in),256)
ourResult2 = decodeUCoordinate(G.elligator2(etc2_in),256)

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


In [2]:

H = H_SHA512()
G = G_X25519()

generate_test_vector(H,G, file=sys.stdout)


##  Test vector for CPace using group X25519 and hash SHA-512


###  Test vectors for calculate_generator with group X25519

~~~
  Inputs
    H   = SHA-512 with input block size 128 bytes.
    PRS = b'Password' ; ZPAD length: 118 ; DSI = b'CPace255'
    CI = b'\nAinitiator\nBresponder'
    CI = 0a41696e69746961746f720a42726573706f6e646572
    sid = 7e4b4791d6a8ef019b936c79fb7f2c57
  Outputs
    hash generator string: (length: 32 bytes)
      8094a6b3638fb04e81fd8fa41c14a12d275da121b271836435f13eac
      7f36c081
    decoded field element of 255 bits: (length: 32 bytes)
      8094a6b3638fb04e81fd8fa41c14a12d275da121b271836435f13eac
      7f36c001
    generator g: (length: 32 bytes)
      2d42aaeeafc98341112f349009438500ccbfc3a468af3ee703538736
      00266459
~~~

###  Test vector for MSGa

~~~
  Inputs
    ADa = b'ADa'
    ya (little endian): (length: 32 bytes)
      21b4f4bd9e64ed355c3eb676a28ebedaf6d8f17bdc365995b3190971
      53044080
  Outputs
    Ya: (length: 32 bytes)
      bda9c0

In [ ]:
H = H_SHAKE256()
G = G_X448()
generate_test_vector(H,G)

In [ ]:
# Test vectors for the C Code, Elligator2

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))


In [ ]:
#

In [ ]:
#8.) Test vector output for ANSI-C, CPace




In [ ]:
import binascii
import random
import hashlib
import sys



prime = 2^255 - 19
F = GF(prime)
A = F(486662)
E = EllipticCurve(F, [0, A , 0, 1 , 0])
point = E.gens()[0]

#print (point_to_octets (point))

#print (point)
#print (octets_to_point(point_to_octets (point),E))

#print (p256_sswu_nu("Hallo.2"))

G_P256 = G_ShortWeierstrass(p256_sswu_nu)


def ByteArrayToInteger(k,numBytes=32):
    try:
        k_list = [ord(b) for b in k]
    except:
        k_list = [b for b in k]
 
    if numBytes < len(k_list):
        numBytes = len(k_list)

    return sum((k_list[i] << (8 * i)) for i in range(numBytes))


scalar = G_P256.sample_scalar()
#scalar = b"\0"
generator = G_P256.calculate_generator(H_SHA256(),b"PRS", b"ci",b"sid", False)
Y = G_P256.scalar_mult(scalar, generator)

#print (G_P256.octets_to_point(Y))

K = G_P256.scalar_mult_vfy(scalar, Y)

print (scalar)
print (generator)
print (Y)
print (K)

In [ ]:
H = H_SHA256()
G = G_ShortWeierstrass(p256_sswu_nu)
#generate_test_vector(H,G)

H = H_SHA384()
G = G_ShortWeierstrass(p384_sswu_nu)
generate_test_vector(H,G)

H = H_SHA512()
G = G_ShortWeierstrass(p521_sswu_nu)
#generate_test_vector(H,G)



In [16]:
import binascii
import random
import hashlib
import sys


H = H_SHA512()
G = G_CoffeeEcosystem(Ed25519Point)

H = H_SHAKE256()
G = G_CoffeeEcosystem(Ed448GoldilocksPoint)
#generate_test_vector(H,G)


P255 = Ed25519Point()

P448 = Ed448GoldilocksPoint()

Ed448GoldilocksPoint.decode((P448.map(H_SHA512().hash(b"1234")) * 0).encode())
Ed25519Point.decode((P255.map(H_SHA512().hash(b"1234")) * 0).encode())
P255 = Ed25519Point()
Ed448GoldilocksPoint.decode(b"\0"*56)*100

Ed448GoldilocksPoint(0x0,0x1)

In [6]:
s = b""
while True:
    s = H_SHAKE256().hash(s,56)
    if (s[0] & 3 == 3):
        break;

print (s)


b'\xaf\x8a\x14!\x8b\xf2\xa2\x06)&\xd2\xea\x9b\x8f\xe4\xe8\xb6\x81sI\xb6\xed/\xeb\x1e]d\xd7\xa4R?\x15\xfc\xee\xc7\x0f\xb1\x11\xe8p\xdcX\xd1\x91\xe6j\x14\xd3\xe9\xd4\x82\xd0D2\xca\xdd'
Input point is on the twist! 


In [11]:
u_twist = H_SHAKE256().hash(b" point on twist ",56)
res_twist = X448(s,u_twist,warnForPointOnTwist = True)


Input point is on the twist! 


In [59]:

str(b"123")

"b'123'"

In [ ]:

with open('testvectors.txt', 'w') as f:

    generate_testvectors_string_functions(file = f)
    
    print("# Test vectors", file = f)
    
    output_test_vectors_for_weak_points_255(file = f)
    H = H_SHA512()
    G = G_X25519()
    generate_test_vector(H,G, file=f)
 
    output_test_vectors_for_weak_points_448(file = f)
    H = H_SHAKE256()
    G = G_X448()
    generate_test_vector(H,G, file=f)
   
    H = H_SHA512()
    G = G_CoffeeEcosystem(Ed25519Point)
    generate_test_vector(H,G, file=f)

    H = H_SHAKE256()
    G = G_CoffeeEcosystem(Ed448GoldilocksPoint)
    generate_test_vector(H,G, file=f)
    
    H = H_SHA256()
    G = G_ShortWeierstrass(p256_sswu_nu)
    generate_test_vector(H,G, file=f)

    H = H_SHA384()
    G = G_ShortWeierstrass(p384_sswu_nu)
    generate_test_vector(H,G, file=f)

    H = H_SHA512()
    G = G_ShortWeierstrass(p521_sswu_nu)
    generate_test_vector(H,G, file=f)
    

    

In [ ]:
H = H_SHA512()
G = G_X25519()
generate_test_vector(H,G)


In [ ]:
values = H_SHA256().hash(b"1234")
print (values,"\n")
print (ByteArrayToLEPrintString(values),"\n")
print (ByteArrayToCInitializer(values,"values"),"\n")

In [ ]:
print (w)

In [ ]:
IntegerToLEPrintString(0x232527dee2cfde76fb425b6d88818630eea7ea263fac28d89f52d096c563b1e)

In [12]:
G_P256 = G_ShortWeierstrass(p256_sswu_nu)


In [31]:

output_weierstrass_invalid_point_test_cases(G_P256)


### Test case for scalar_mult_vfy with correct inputs

~~~
    y: (length: 32 bytes)
      f012501c091ff9b99a123fffe571d8bc01e8077ee581362e1bd21399
      0835643b
    X: (length: 65 bytes)
      04d0562b1f0126184d3fcb9fd40e2ce5d98f28cc73dcdc1180bf311b
      4be915208e658cb60cdb191afd34af40053710280d67909d26bd510d
      9806d0c6ba36f9b991
    G.scalar_mult_vfy(y,X): (length: 32 bytes)
      354d409cc6c5f6ec375a8c4b22cdcf985e2aac21d8a65d7b964dbc1f
      fb80a5bc
~~~

### Invalid inputs for scalar_mult_vfy which MUST result in aborts
For these test cases scalar_mult_vfy(y,.) MUST return the representation of the neutral element G.I. A G.I result from scalar_mult_vfy MUST make the protocol abort!.
~~~
    s: (length: 32 bytes)
      f012501c091ff9b99a123fffe571d8bc01e8077ee581362e1bd21399
      0835643b
    Y_i1: (length: 65 bytes)
      04d0562b1f0126184d3fcb9fd40e2ce5d98f28cc73dcdc1180bf311b
      4be915208e658cb60cdb191afd34af40053710280d67909d26bd510d
      9806d0c6ba36f9b9b8
    Y_i2

In [57]:
H = H_SHA512()
G = G_CoffeeEcosystem(Ed25519Point)
X = G.calculate_generator( H, b"Password", b"CI", b"sid")
y = G.sample_scalar(deterministic_scalar_for_test_vectors= b"yes we want it")
Y = G.scalar_mult(y,X)
Y_wrong = bytearray(Y)
Y_wrong[3] += 1
K = G.scalar_mult_vfy(y,Y_wrong)

def output_coffee_invalid_point_test_cases(G, file = sys.stdout):
    X = G.calculate_generator( H_SHAKE256(), b"Password", b"CI", b"sid")
    y = G.sample_scalar(deterministic_scalar_for_test_vectors= b"yes we want it")
    K = G.scalar_mult_vfy(y,X)
    Z = G.scalar_mult(y,X)
    print ("\n### Test case for scalar_mult with valid inputs\n", file = file)
    print ("~~~", file = file)
    tv_output_byte_array(y, test_vector_name = "s", 
                         line_prefix = "    ", max_len = 60, file = file)
    tv_output_byte_array(X, test_vector_name = "X", 
                         line_prefix = "    ", max_len = 60, file = file)
    tv_output_byte_array(Z, test_vector_name = "G.scalar_mult(s,decode(X))", 
                         line_prefix = "    ", max_len = 60, file = file)
    tv_output_byte_array(K, test_vector_name = "G.scalar_mult_vfy(s,X)", 
                         line_prefix = "    ", max_len = 60, file = file)
    print ("~~~\n", file = file)
    
    Y_inv1 = bytearray(X)
    for m in range(16*256):
        Y_inv1[m%16] = (Y_inv1[m%16] - 1) % 256 # choose an incorrect value    
        K_inv1 = G.scalar_mult_vfy(y,Y_inv1)
        if K_inv1 == G.I:
            break
                   
    print ("\n### Invalid inputs for scalar_mult_vfy which MUST result in aborts\n", file = file)
    print ("For these test cases scalar_mult_vfy(y,.) MUST return the representation"+
           " of the neutral element G.I. A G.I result from scalar_mult_vfy MUST make" +
           " the protocol abort!.", file = file)
    print ("~~~", file = file)
    tv_output_byte_array(y, test_vector_name = "s", 
                         line_prefix = "    ", max_len = 60, file = file)
    tv_output_byte_array(Y_inv1, test_vector_name = "Y_i1", 
                         line_prefix = "    ", max_len = 60, file = file)   
    tv_output_byte_array(G.I, test_vector_name = "G.I", 
                         line_prefix = "    ", max_len = 60, file = file)
    print ("    G.scalar_mult_vfy(s,Y_i1) = G.scalar_mult_vfy(s,G.I) = G.I", file = file)
    print ("~~~\n", file = file)    

output_coffee_invalid_point_test_cases(G)


### Test case for scalar_mult with valid inputs

~~~
    s: (length: 32 bytes)
      7cd0e075fa7955ba52c02759a6c90dbbfc10e6d40aea8d283e407d88
      cf538a05
    X: (length: 32 bytes)
      740da933b82592280ed7219789f4bb53a50e94aa58971d4b677a0a4a
      a56a9850
    G.scalar_mult(s,decode(X)): (length: 32 bytes)
      02d48918716a6c443fa89f19c7541065518c689d984f8c08758c0de4
      7f02361c
    G.scalar_mult_vfy(s,X): (length: 32 bytes)
      02d48918716a6c443fa89f19c7541065518c689d984f8c08758c0de4
      7f02361c
~~~


### Invalid inputs for scalar_mult_vfy which MUST result in aborts

For these test cases scalar_mult_vfy(y,.) MUST return the representation of the neutral element G.I. A G.I result from scalar_mult_vfy MUST make the protocol abort!.
~~~
    s: (length: 32 bytes)
      7cd0e075fa7955ba52c02759a6c90dbbfc10e6d40aea8d283e407d88
      cf538a05
    Y_i1: (length: 32 bytes)
      730da933b82592280ed7219789f4bb53a50e94aa58971d4b677a0a4a
      a56a9850
    G.I: (length: 32 bytes)
 

In [49]:
("begin").title()

'Begin'

In [6]:
import sys
file = sys.stdout
print ('    b"\\0" > b"\\0\\0" ==', b"\0" > b"\0\0", file = file)
print ('    b"\\1" > b"\\0\\0" ==', b"\1" > b"\0\0", file = file)
print ('    b"\\0\\0" > b"\\0" ==', b"\0\0" > b"\0", file = file)
print ('    b"\\0\\0" > b"\\1" ==', b"\0\0" > b"\1", file = file)
print ('    b"\\0\\1" > b"\\1" ==', b"\0\1" > b"\1", file = file)
print ('    b"ABCD" > b"BCD" ==', b"ABCD" > b"BCD", file = file)

def lexiographically_larger(bytes1,bytes2):
    min_len = min (len(bytes1), len(bytes2))
    for m in range(min_len):
        if bytes1[m] > bytes2[m]:
            return True;
        elif bytes1[m] < bytes2[m]:
            return False;
    return len(bytes1) > len(bytes2)

ll = lexiographically_larger
print("")
print ('    b"\\0" LL b"\\0\\0") ==', ll(b"\0",b"\0\0"), file = file)
print ('    b"\\1" LL b"\\0\\0" ==', ll(b"\1", b"\0\0"), file = file)
print ('    b"\\0\\0" LL b"\\0" ==', ll(b"\0\0", b"\0"), file = file)
print ('    b"\\0\\0" LL b"\\1" ==', ll(b"\0\0", b"\1"), file = file)
print ('    b"\\0\\1" LL b"\\1" ==', ll(b"\0\1", b"\1"), file = file)
print ('    b"ABCD" LL b"BCD" ==', ll(b"ABCD", b"BCD"), file = file)



    b"\0" > b"\0\0" == False
    b"\1" > b"\0\0" == True
    b"\0\0" > b"\0" == True
    b"\0\0" > b"\1" == False
    b"\0\1" > b"\1" == False
    b"ABCD" > b"BCD" == False

    b"\0" LL b"\0\0") == False
    b"\1" LL b"\0\0" == True
    b"\0\0" LL b"\0" == True
    b"\0\0" LL b"\1" == False
    b"\0\1" LL b"\1" == False
    b"ABCD" LL b"BCD" == False
